<a href="https://colab.research.google.com/github/joaqsluix/acmdigw/blob/main/EXTRACTING_AND_SAVING_DATA_ACMDIGW22624.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("test")

test


In [2]:
# this is a comment
1+1

2

In [3]:
# most common shortcut
# ctrl + / = comment a code
# ctrl + enter = run a code block
# ctrl + m, b = create a code block below
# ctrl + m, a = create a code block above

In [4]:
# connect gdrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# install yelpapi
!pip install yelpapi

In [6]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# additional libraries

import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

LOADING CREDENTIALS

In [8]:
# loading credentials and Creating the YelpAPI object
# load yelp API credentials and instantiate the YelpAPI object
with open('/content/drive/MyDrive/Colab Notebook/credentials/yelp_api.json') as file:
  yelp_credentials = json.load(file)

# create the yelp api engine
yelp_api = YelpAPI(yelp_credentials['api-key'], timeout_s = 5.0)

In [12]:
# define API call parameters and output file path
LOCATION = 'Greenville, SC'
TERM = 'Sushi'
JSON_FILE ='/DATA/results_SC_sushi.json'

# display the destination file path
print(f'Data will be saved to:{JSON_FILE}')

Data will be saved to:/DATA/results_SC_sushi.json


In [16]:
# check if JSON_FILE exists and create if it doesn't
if not os.path.isfile(JSON_FILE):

  # create the directory
  os.makedirs(os.path.dirname(JSON_FILE), exist_ok = True)

  # confirm and save an empty list on the file
  print(f'[i]{JSON_FILE} not found. Saving empty list on file')
  with open(JSON_FILE,'w') as file:
        json.dump([], file)

else:
    # inform the user if the file already exists
    print(f'[i]{JSON_FILE} already exists.')

[i]/DATA/results_SC_sushi.json not found. Saving empty list on file


In [17]:
# load previous results and set offset based on the number of results
with open(JSON_FILE, 'r') as file:
  previous_results =json.load(file)

n_results = len(previous_results)

print(f'{n_results} previous results found.')

0 previous results found.


In [19]:
# make first API call
results = yelp_api.search_query(location = LOCATION,
                                term = TERM,
                                offset = n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [20]:
# return how many results were found
total_results = results['total']
total_results

111

In [21]:
business_data = results['businesses']

# specify the file name where you want to save the data
json_file_path = JSON_FILE

# save the business data
with open(json_file_path, 'w') as file:
  json.dump(business_data, file, indent = 4)